In [1]:
## 
using Revise
includet("../src/AeroMDAO.jl")
includet("../src/MathTools.jl")

In [2]:
##
using .AeroMDAO
using .MathTools: linspace, tuparray
using DelimitedFiles
using Rotations

In [5]:
## Wing section setup
foilpath = "../airfoil_database/ys930.dat"

num_secs = 3

coords = read_foil(foilpath)
foils = [ coords for i in 1:num_secs ]
airfoils = Foil.(foils)

wing_chords = [1, 1, 0.2]
wing_twists = [0, 0, 0]
wing_spans = [2, 1]
wing_dihedrals = [0, 45]
wing_sweeps = [0, 30]

wing_right = HalfWing(airfoils, wing_chords, wing_spans, wing_dihedrals, wing_sweeps, wing_twists)
wing = Wing(wing_right, wing_right)
print_info(wing_right)

Span: 2.7071067811865475 m
Area: 2.3674234614174767 m²
MAC: 0.9282051282051281 m
Aspect Ratio: 3.0955286386992


Assembly

In [10]:
wing_panels = make_panels(wing, spanwise_panels = 5, chordwise_panels = 1)
test_panels = mesh_wing(wing, spanwise_panels = 1, chordwise_panels = 15)

112-element Array{Panel3D,1}:
 Panel3D([1.0, 0.0, 0.0], [0.9874640814513509, 0.0, 0.0030573025143876263], [0.9874640814513509, -2.0, 0.0030573025143876263], [1.0, -2.0, 0.0])
 Panel3D([0.9874640814513509, 0.0, 0.0030573025143876263], [0.9504849291068701, 0.0, 0.013105024874003167], [0.9504849291068701, -2.0, 0.013105024874003167], [0.9874640814513509, -2.0, 0.0030573025143876263])
 Panel3D([0.9504849291068701, 0.0, 0.013105024874003167], [0.8909168320766025, 0.0, 0.026017878523143825], [0.8909168320766025, -2.0, 0.026017878523143825], [0.9504849291068701, -2.0, 0.013105024874003167])
 Panel3D([0.8909168320766025, 0.0, 0.026017878523143825], [0.8117467834803576, 0.0, 0.042056703937146105], [0.8117467834803576, -2.0, 0.042056703937146105], [0.8909168320766025, -2.0, 0.026017878523143825])
 Panel3D([0.8117467834803576, 0.0, 0.042056703937146105], [0.7169447001400835, 0.0, 0.058954358082818015], [0.7169447001400835, -2.0, 0.058954358082818015], [0.8117467834803576, -2.0, 0.0420567039371461

Panel case

In [7]:
ρ = 1.225
uniform = Uniform(10.0, 5.0, 0.0)
@time lift, drag = solve_case(wing_panels, uniform, ρ)

V = uniform.mag
S = projected_area(wing_right)
cl = lift/(ρ/2 * V^2 * S)
cdi = drag/(ρ/2 * V^2 * S)
println("Lift: $(sum(lift)), Drag: $(sum(drag))")
println("Lift Coefficient: $(sum(cl)), Drag Coefficient: $(sum(cdi))")
println("Lift-to-Drag Ratio (L/D): $(sum(cl)/sum(cdi))")

  8.811067 seconds (16.55 M allocations: 862.228 MiB, 8.27% gc time)
Lift: 14.430150629002176, Drag: 1.1033570656737903
Lift Coefficient: 0.09951506345326523, Drag Coefficient: 0.007609113114970189
Lift-to-Drag Ratio (L/D): 13.078405058465885


In [9]:
using Plots, LaTeXStrings

In [ ]:
plotlyjs()

┌ Info: Precompiling PlotlyJS [f0f68f2c-4968-5e81-91da-67840de0976a]
└ @ Base loading.jl:1278
┌ Warning: Module MbedTLS with build ID 115046336255914 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1017
┌ Warning: Module MbedTLS with build ID 115046336255914 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1017
┌ Warning: Module MbedTLS with build ID 115046336255914 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1017
┌ Warning: Module MbedTLS with build ID 115046336255914 is missing from the cache.
│ This may mean MbedTLS [739be429-bea8-5141-9913-cc70e7f3736d] does not support precompilation bu

Wing Plot

In [ ]:
# wing_plot = plot_setup(coordinates(wing))
# wing_sects = plot_setup.(sections(wing))
# wing_pts = horseshoe_vortex.(wing_panels)
wing_collocs = horseshoe_collocation.(wing_panels)
pan_coords = (tuparray ∘ panel_coords).(test_panels)
spans = [ pt[2] for pt in wing_collocs ]

plot(spans, cl)

##
plot(spans, cdi)

##
plot(xaxis = "x", yaxis = "y", zaxis = "z")
plot!.(pan_coords, line = :dash, color = :black, label = :none)

plot!.(wing_pts, c = :black, label = :none)

scatter!(wing_collocs, c = :grey, label = "Wing Collocation Points")

# wing_lines = [ horseshoe_lines(panel, uniform) for panel in wing_panels ]

# lines = [ [ tuparray([line.r1'; line.r2']) for line in horseshoe.vortex_lines ] for horseshoe in wing_lines ]
# [ [ plot!(line, c = :darkblue, label = :none) for line in horseshoe ] for horseshoe in lines ]

gui();